# Convert Word (Transcript) to DataFrame

Transcription using MS Word creates a format where timestamp and speaker are on one line, and the utterance is on the next. This notebook converts this format (from a Word docx file) to a CSV or Excel file by first converting it into a Pandas DataFrame first.

In [225]:
import docx2txt
import pandas as pd
from IPython.display import display, Markdown

### Get relevant text from the Word Doc
The Word document often contains some preamble, typically followed by the word `Transcript` after which the transcript begins. Test this assumption by printing the contents of the variable `text` below. If this is correct, then only extract the text _after_ the word 'Transcript'.

In [260]:
transcript_path = "data/mia-transcript.docx"
transcript = docx2txt.process(transcript_path)
# print the first 500 characters, just to check.
transcript[:500]

"T\n\nBoth. But you have to record everything.  \n\nM\n\nNice. I'm Mia. Nice to meet you. We know each other. This is Bas. Who Is I don't know What. \n\nB\n\nI'm doing something important.\n\nM\n\nOK whatever\n\nT\n\nDo you know any quiet places here at the faculty?\n\nM\n\nwe could do, we could get, like a meeting room and then go like we can get that meeting room over there.\n\nR\n\nIn the project room maybe? \n\nT\n\nOhh yeah, we can just steal one.\n\nM\n\nOh yes, we can check the project rooms. But I feel like those were alw"

### Clean up the text
A number of carriage returns will be in the text file. This is useful to split up the file into lines, so the code below does the following:
1. Splits the text into lines where a line is either a `timestamp, speaker` combo an `utterance`, or an empty line (in case of multiple successive carriage returns;
2. Removes all empty lines;
3. Replaces all non-breaking spaces (unicode: `\xa0`)
4. Splits the `'timestamp, speaker'` string combination into a list with `['timestamp', 'speaker']` in it.

In [261]:
transcript_list = transcript.split("\n")
transcript_list = [s.strip() for s in transcript_list if len(s.strip()) > 0]
transcript_list = [s.replace('\xa0',  ' ') for s in transcript_list]
transcript_list

['T',
 'Both. But you have to record everything.',
 'M',
 "Nice. I'm Mia. Nice to meet you. We know each other. This is Bas. Who Is I don't know What.",
 'B',
 "I'm doing something important.",
 'M',
 'OK whatever',
 'T',
 'Do you know any quiet places here at the faculty?',
 'M',
 'we could do, we could get, like a meeting room and then go like we can get that meeting room over there.',
 'R',
 'In the project room maybe?',
 'T',
 'Ohh yeah, we can just steal one.',
 'M',
 'Oh yes, we can check the project rooms. But I feel like those were always full.',
 'T',
 'I thought you were English.',
 'M',
 "I am but I'm learning Dutch.",
 'R',
 'Really?',
 'M',
 'Yeah. Are you Spanish? Yeah.',
 'R',
 'Yeah, really evident.',
 'T',
 'Were you the one who were posting those super views?',
 'M',
 'Ohh my gosh, yes.',
 'T',
 "I'm so upset that they're no longer there because someone complained.",
 'M',
 "I am very upset myself as well. I left. I left the chat. I'm done.",
 'T',
 'No, serious?',
 '

In [262]:
print("Remove last item if it is incomplete.")
print('--------------------------- LAST 4 LINES ---------------------------')
for t in transcript_list[-4:] :
    print(t)
print('-------------------------------- END -------------------------------')

# Uncomment and re-run this cell if the last line is found to be incomplete
# if type(transcript_list[-1]) == list :
    # transcript_list.pop() 

Remove last item if it is incomplete.
--------------------------- LAST 4 LINES ---------------------------
T
Yes
M
I think we should stop that.
-------------------------------- END -------------------------------


### Convert into DataFrame
Conver the list with successive items being `['timestamp', 'speaker]` and `'utterance'` items into a DataFrame

In [263]:
transcript_triad = []
for ind, line in enumerate(transcript_list) :
    if ind % 2 == 0 and ind+1 < len(transcript_list):
        # these are speaker/timestamp lines
        if line[0].isdigit() :
            # the start is a timestamp
            timestamp = line.split(' ')[0]
            speaker = ' '.join(line.split(' ')[1:])
            utterance = transcript_list[ind+1]
        else :
        # The line contains speaker but not timestamps 
            timestamp = ''
            speaker = line
            if len(line.split(' ')) > 2 :
                print("*****************************************")
                print("Problem found!")
                print(transcript_list[ind-3:ind+1])
                print(ind)
                print("*****************************************")
                break
            utterance = transcript_list[ind+1]
        turn_data = { 'timestamp' : timestamp,
                      'speaker'   : speaker,
                      'utterance' : utterance
                    }
        transcript_triad.append(turn_data)
            

In [264]:
# print(transcript_triad)
df = pd.DataFrame(transcript_triad)
df.sample(5)

,timestamp,speaker,utterance
318,,M,It is quite spacious in there.
299,,M,Ohh that's cute.
549,,B,"Well, it's sort of funny because it's what eve..."
451,,T,"Yes. Well, they've got a MacBook, so they have..."
47,,M,"Ohh my gosh, yes I saw."


### Save DataFrame
Because this involves text which can have punctuations including commas and semicolons, a CSV file is not recommended. Instead, save it as an excel file.

In [265]:
excel_name = transcript_path.split(".docx")[0] + ".xlsx"
df.to_excel(excel_name)
print("File saved to:", excel_name)

File saved to: data/mia-transcript.xlsx
